# CH08.Building image

### 1. Echo-server with python:3.8.2
간단하게 python:3.8.2 환경으로 echo-server를 작성하고 다음과 같이 디렉토리를 구성하자.
- ./myservice/echo_server.py

```nc```를 이용하여 정상작동하는지 확인하고 dockerfile을 구성한다.

In [12]:
!python3 ../src/building_image/myservice/echo_server.py

server is started
Connected by ('127.0.0.1', 62665)
Message by  ('127.0.0.1', 62665)  :  b'Hello world!\n'


### 2. Dockerfile and build
Docker image를 만들려면 dockerfile이 필요하다. dockerfile이란 docker iamge를 만들기위한 설정 파일이며, 여러가지 명령어를 토대로 dockerfile을 작성하면, 설정된 내용대로 docker image를 만들 수 있다. 즉, dockerfile을 읽을 수 있다는 말은 해당 이미지가 어떻게 구성되어있는지 알 수 있다는 말이다.<br><br>

관련된 문법과 자세한 설명은 docker official docs에서 확인가능하다. <br><br>

먼저 dir 구성을 다음과 같이한다. 필요한 데이터는 무조건 dockerfile의 하위 디렉토리에 담아준다.
- ./myservice/dockerfile
- ./myservice/echo_server.py

__dockerfile__
```dockerfile
FROM python:3.8.2

RUN mkdir /echo
COPY echo_server.py /echo

CMD ["python", "/echo/echo_server.py"]
```

먼저 ```FROM```은 base image를 선택해주는 것이다. ```RUN```은 image를 생성하며 실행하는 command이고, <strong>/echo</strong>라는 디렉토리를 생성하였다. 추가적으로 ```COPY```를 통해서 myservice안에 있는 <strong>echo_server.py</strong>를 image의 <strong>/echo</strong>로 복사하는 명령을 하였고, ```CMD``` 명령은 Build할 때가 아닌, container를 실행할 때, 자동으로 프로그램을 실행하도록 하는 설정이다. 이는 dockerfile에서 한번만 언급될 수 있다.<br><br>

이렇게 dockerfile을 구성하였으면, 다음과 같은 명령어로 image를 Build할 수 있다.<br>
```docker build -t myservice [DIR_PATH]```<br>
\[DIR_PATH\]가 가리키는 곳에는 항상 dockerfile이 있어야한다.

```shell
 ~/Desktop/Learning-Container/src/building_image/myservice $ docker pull python:3.8.2
3.8.2: Pulling from library/python
...
Status: Downloaded newer image for python:3.8.2
docker.io/library/python:3.8.2
 ~/Desktop/Learning-Container/src/building_image/myservice $ cat > dockerfile
FROM python:3.8.2

RUN mkdir /echo
COPY echo_server.py /echo

CMD ["python", "/echo/echo_server.py"]
 ~/Desktop/Learning-Container/src/building_image/myservice $ docker build -t myservice .
[+] Building 1.1s (8/8) FINISHED                                                                                                               
 => [internal] load build definition from Dockerfile                                                                                      0.1s
 => => transferring dockerfile: 162B                                                                                                      0.0s
 => [internal] load .dockerignore                                                                                                         0.0s
 => => transferring context: 2B                                                                                                           0.0s
 => [internal] load metadata for docker.io/library/python:3.8.2                                                                           0.0s
 => [1/3] FROM docker.io/library/python:3.8.2                                                                                             0.1s
 => => resolve docker.io/library/python:3.8.2                                                                                             0.0s
 => [internal] load build context                                                                                                         0.1s
 => => transferring context: 519B                                                                                                         0.0s
 => [2/3] RUN mkdir /echo                                                                                                                 0.7s
 => [3/3] COPY echo_server.py /echo                                                                                                       0.0s
 => exporting to image                                                                                                                    0.0s
 => => exporting layers                                                                                                                   0.0s
 => => writing image sha256:a8305c3**************************************************510811e                                              0.0s
 => => naming to docker.io/library/myservice                                                                                              0.0s
 ~/Desktop/Learning-Container/src/building_image/myservice $ docker images
REPOSITORY   TAG       IMAGE ID       CREATED         SIZE
myservice    latest    a8305c*****b   3 minutes ago   934MB
python       3.8.2     4f7cd4*****9   9 months ago    934MB
```
<br>

Container를 만들어서 실행시켜서 테스트를 해보면 다음과 같다.
```shell
 ~/Desktop/Learning-Container/src/building_image/myservice $ docker run -d -p 50000:50000 --name echo_server --rm myservice
1d1f*******************************************************b
 ~/Desktop/Learning-Container/src/building_image/myservice $ nc 127.0.0.1 50000
Hello world!
Hello world!
^C
 ~/Desktop/Learning-Container/src/building_image/myservice $ docker run -t -p 60000:50000 --name echo_server --rm myservice
server is started
^CTraceback (most recent call last):
  File "/echo/echo_server.py", line 10, in <module>
    client_socket, client_addr = server_socket.accept()
  File "/usr/local/lib/python3.8/socket.py", line 292, in accept
    fd, addr = self._accept()
KeyboardInterrupt
```

<br>
물론 dockerfile에서 사용할 수 있는 추가적인 키워드가 존재한다. 내용은 다음과 같다.

- ```WORKDIR [PATH]``` : Docker 이미지 내부에서 RUN, CMD, ENTRYPOINT의 명령이 실행될 디렉토리를 설정한다.
- ```ADD [PATH]``` : 현재 디렉터리에 있는 파일들을 이미지 내부 \[PATH\] 디렉토리에 추가한다.

VOLUME ["/data", "/var/log/httpd"]
- ```EXPOSE [PORT]``` : 호스트와 연결시킬 port를 설정한다.
- ```WORKDIR [PATH]``` : Docker 이미지 내부에서 RUN, CMD, ENTRYPOINT의 명령이 실행될 디렉터리를 설정한다.
- ```WORKDIR [PATH]``` : Docker 이미지 내부에서 RUN, CMD, ENTRYPOINT의 명령이 실행될 디렉터리를 설정한다.
- ```WORKDIR [PATH]``` : Docker 이미지 내부에서 RUN, CMD, ENTRYPOINT의 명령이 실행될 디렉터리를 설정한다.
- ```WORKDIR [PATH]``` : Docker 이미지 내부에서 RUN, CMD, ENTRYPOINT의 명령이 실행될 디렉터리를 설정한다.
- ```WORKDIR [PATH]``` : Docker 이미지 내부에서 RUN, CMD, ENTRYPOINT의 명령이 실행될 디렉터리를 설정한다.
- ```WORKDIR [PATH]``` : Docker 이미지 내부에서 RUN, CMD, ENTRYPOINT의 명령이 실행될 디렉터리를 설정한다.
- ```WORKDIR [PATH]``` : Docker 이미지 내부에서 RUN, CMD, ENTRYPOINT의 명령이 실행될 디렉터리를 설정한다.


### 3. Image 경량화
위에서 본 내용들은 사실상 정론에 가까운 내용들이고, 실제로 이렇게 사용하면 문제가 많다. dockerfile을 작성하는 방법은 매우 다양하고 어떻게 하든 image가 만들어지면 배포 할 수 있다. 그래서 중요한게 최적화다. 잘 작성된 dockerfile은 image 사이즈를 줄이고 빌드/배포 시간을 단축시킨다. 위의 내용중 ```docker images```를 통해서 방금 만든 image의 사이즈를 보면 다음과 같다.

```shell
REPOSITORY   TAG       IMAGE ID       CREATED         SIZE
myservice    latest    a8305c*****b   3 minutes ago   934MB
python       3.8.2     4f7cd4*****9   9 months ago    934MB
```
이 부분을 참고하면 python 환경에서 python 프로그램을 얹은 image 파일이 사이즈가 934MB이다. 사실상 경량화가 실패한 image이고, 이는 꽤나 치명적이다. docker image를 구성할 때(dockerize) 필요한 체크리스트는 다음과 같다. 알파인, 파이썬은 데비안-부스터 (python:3.8-buster 또는 3.8-slim-buster)


#### 3.1. Use minimal base image
 베이스 이미지를 선택할 때, 작은 사이즈의 이미지를 사용해야한다. 굉장히 당연한 말이고, 일반적으로 사이즈가 작은 alpine-linux를 사용한다.

In [16]:
!docker search alpine

NAME                                   DESCRIPTION                                     STARS     OFFICIAL   AUTOMATED
alpine                                 A minimal Docker image based on Alpine Linux…   7149      [OK]       
mhart/alpine-node                      Minimal Node.js built on Alpine Linux           481                  
anapsix/alpine-java                    Oracle Java 8 (and 7) with GLIBC 2.28 over A…   468                  [OK]
frolvlad/alpine-glibc                  Alpine Docker image with glibc (~12MB)          254                  [OK]
alpine/git                             A  simple git container running in alpine li…   165                  [OK]
mvertes/alpine-mongo                   light MongoDB container                         117                  [OK]
yobasystems/alpine-mariadb             MariaDB running on Alpine Linux [docker] [am…   83                   [OK]
alpine/socat                           Run socat command in alpine container           66          

 그치만 이 또한 상황에 맞게 사용해야한다. 예를 들어서 python 어플리케이션의 경우는 alpine-linux를 사용하는 것에 적합하지 않다. PyPI에 올라간 파이썬 라이브러리들은 보통 wheel 포맷을 사용하는 데 alpine-linux는 wheel 포맷을 지원하지 않는다. alpine-linux를 사용한다면 python 패키지에서 C 코드를 컴파일 해야 하므로 image를 빌드하는 시간이 상당히 많이 소모된다. 그래서 python의 경우는 Debian Buster를 기반으로 한 ```python:3.8-buster``` 또는 ```3.8-slim-buster```를 사용하는 것이 좋다.

#### 3.2. Reduce number of image layers
레이어는 ```RUN```, ```ADD```, ```COPY``` 명령에서만 생성된다.
레이어 개수가 적다고 도커 이미지/컨테이너 성능에 영향을 주진 않지만 Dockerfile 가독성과 유지 보수 관점에서 도움이 될 것이다.

```dockerfile
RUN apt-get update
RUN apt-get -y install git
RUN apt-get -y install locales
RUN apt-get -y install gcc
```
위의 경우는 4개의 layer가 생성되고, 다음과 같이 chaning할 수 있다.

```dockerfile
RUN apt-get update && apt-get install -y \
    gcc \
    git \
    docker
```
이 경우는 layer를 1개로 줄일 수 있다.


#### 3.3. Move user application code to below
Python에서 일반적으로 사용되는 패키지 관리자인 pip의 경우는 관행적으로 ```requirement.txt```파일로 의존성 패키지를 명시한다. 이는 ```pip freeze > requirement.txt``` 명령으로 얻을 수 있고, 다음과 같은 명령으로 설치하는 것이 일반적이다. ```pip install -r requirement.txt```. 문제는 이러한 의존성 패키지에 대한 문제점이 캐시가 될 때 발생한다. 다음과 같이 파일을 구성하면 이를 막을 수 있다.

```dockerfile
FROM python:3.8-slim-buster

WORKDIR /workspace

COPY requirements.txt /workspace
COPY myservice /workspace

RUN pip install -r requirement.txt

CMD ["pip", "freeze"]
```
상대적으로 많이 리캐시가 일어나는 myservice를 밑에 두면, 의존성파일에 대한 내용은 초기화가 잘 일어나지 않는다. 즉, User application code의 COPY 명령은 자주 변경되지 않는 명령문 다음에 오는 것이 빌드 시간을 단축하는 데 합리적이다.


#### 3.4. Multi-stage build
Multi-stage build란 image의 빌드에는 필요하지만 최종 image에는 필요 없는 환경을 제거할 수 있도록 단계를 나누어서 image를 만드는 방법이다.
일단 최종적으로 python 어플리케이션을 돌릴 수 있는 가장 최적화된 이미지는 다음과 같다.

```dockerfile
# Build stage
FROM python:3.8.2-slim-buster AS pip
RUN apt-get update
RUN apt-get install -y --no-install-recommends build-essential gcc
RUN pip install --user --no-warn-script-location [PACKAGE1] [PACKAGE2]

# Executable stage
FROM python:3.8.2-alpine3.11
COPY --from=pip /root/.local /root/.local
ENV PATH=/root/.local/bin:$PATH
```

이렇게 stage를 두 개로 나누어서 build하면 더욱 경량화 할 수 있다. 실제로 이렇게 numpy, matplotlib를 포함하는 python 이미지를 만들어서 확인하면 크게 차이가 난다.

```shell
 ~/Desktop/Learning-Container/src/mypython $ docker images
REPOSITORY   TAG       IMAGE ID       CREATED             SIZE
mypython     latest    64da1c1ea109   8 seconds ago       216MB
myservice    latest    a8305c39c7ab   About an hour ago   934MB
python       3.8.2     4f7cd4269fa9   9 months ago        934MB
```

#### 3.5. Use .dockerignore file
Image를 생성할 때 파일을 모두 docker 데몬에 전송하므로 필요 없는 파일이 포함되지 않도록 해야한다. 이때 .dockerignore 파일을 사용하면 된다. Docker는 Go 언어로 작성되어 있기 때문에 파일 매칭도 Go 언어의 규칙을 따름을 주의해야한다.

__.dockerignore__
```
example/hello.txt
example/*.cpp
wo*
*.cpp
.git
.svn
```

<strong>Reference.</strong><br>
Dockerfile docs : https://docs.docker.com/develop/develop-images/dockerfile_best-practices/<br>
https://pythonspeed.com/articles/alpine-docker-python/<br>
https://pythonspeed.com/articles/base-image-python-docker-images/<br>
https://zetawiki.com/wiki/Python용_멀티스테이지_빌드_Dockerfile<br>